In [ ]:
# Generate refs and mixtures
import pandas as pd
import os

median_range = [15]
sub_range = [1, 5, 10, 15]
distribution = ["crd_dis", "uniform_dis", "dirichlet_dis"]

df = pd.read_table("~/bed_hg38/bed_hg38.bed", header=None)
df.columns = ["chrom", "start", "end", "cpg_idx"]
for dist in distribution:

    for median_value in median_range:

        csv_file = f"/cfDNA_benchmark/cfNOMe_data/{dist}/ref_median_{median_value}/full_atlas_crd_median_{median_value}.csv"

        csv_df = pd.read_csv(csv_file)
        tmp_df = df.copy()

        csv_cpg_idx = set(csv_df["cpg_idx"])
        tmp_df = tmp_df[tmp_df["cpg_idx"].isin(csv_cpg_idx)]
        merged_df = pd.merge(tmp_df, csv_df, on="cpg_idx")
        merged_df = pd.merge(df, csv_df, on="cpg_idx")
        merged_df = merged_df[["chrom", "start", "end"] + csv_df.columns.tolist()]
        out_file = (
            f"ref_median_{median_value}/{dist}/full_atlas_crd_median_{median_value}.bed"
        )
        merged_df.to_csv(out_file, index=False)

    for median_value in median_range:
        csv_file = f"/cfDNA_benchmark/cfNOMe_data/{dist}/ref_median_{median_value}/reference_overlapped_midean_{median_value}_unique.csv"
        csv_df = pd.read_csv(csv_file)
        tmp_df = df.copy()
        csv_cpg_idx = set(csv_df["cpg_idx"])
        tmp_df = tmp_df[tmp_df["cpg_idx"].isin(csv_cpg_idx)]
        merged_df = pd.merge(tmp_df, csv_df, on="cpg_idx")
        merged_df = pd.merge(df, csv_df, on="cpg_idx")
        merged_df = merged_df[["chrom", "start", "end"] + csv_df.columns.tolist()]
        out_file = f"/cfDNA_benchmark/cfNOMe_data/{dist}/ref_median_{median_value}/reference_overlapped_midean_{median_value}_unique.bed"
        merged_df.to_csv(out_file, sep="\t", index=False)

    for median_value in median_range:

        median_dir = f"/{dist}/ref_median_{median_value}"

        df = pd.read_csv(
            os.path.join(median_dir, f"full_atlas_crd_median_{median_value}.bed")
        )

        crd_cols = [col for col in df.columns if col.startswith("crd_")]

        for col in crd_cols:

            new_df = df[["chrom", "start", "end", "cpg_idx", col]]
            col_name = col.split("_")[1]
            out_path = os.path.join(median_dir, f"crd_{col_name}_1.bed")
            new_df.to_csv(out_path, sep="\t", index=False, header=False)